In [0]:
%sql
SELECT 
  result_state,
  COUNT(*) AS count,
  ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS percentage
FROM (
  SELECT 
    CASE 
      WHEN result_state = 'SUCCESS' THEN 'SUCCEEDED'
      WHEN result_state IN ('ERROR', 'FAILED', 'SKIPPED') THEN 'ERROR'
      ELSE result_state
    END AS result_state
  FROM system.lakeflow.job_task_run_timeline
  WHERE period_start_time > (SELECT MIN(change_time) FROM system.lakeflow.jobs)
    AND job_id IN (
      SELECT DISTINCT job_id
      FROM system.lakeflow.jobs
      WHERE change_time > (SELECT MAX(delete_time) + INTERVAL 1 MINUTE FROM system.lakeflow.jobs)
    )
) AS subquery
GROUP BY result_state
ORDER BY count DESC
